# Amazon SageMaker Multi-Model Endpoints using Linear Learner
With [Amazon SageMaker multi-model endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html), customers can create an endpoint that seamlessly hosts up to thousands of models. These endpoints are well suited to use cases where any one of a large number of models, which can be served from a common inference container, needs to be invokable on-demand and where it is acceptable for infrequently invoked models to incur some additional latency. For applications which require consistently low inference latency, a traditional endpoint is still the best choice.

At a high level, Amazon SageMaker manages the loading and unloading of models for a multi-model endpoint, as they are needed. When an invocation request is made for a particular model, Amazon SageMaker routes the request to an instance assigned to that model, downloads the model artifacts from S3 onto that instance, and initiates loading of the model into the memory of the container. As soon as the loading is complete, Amazon SageMaker performs the requested invocation and returns the result. If the model is already loaded in memory on the selected instance, the downloading and loading steps are skipped and the invocation is performed immediately.

Amazon SageMaker inference pipeline model consists of a sequence of containers that serve inference requests by combining preprocessing, predictions and post-processing data science tasks.  An inference pipeline allows you to apply the same preprocessing code used during model training, to process the inference request data used for predictions.

To demonstrate how multi-model endpoints are created and used with inference pipeline, this notebook provides an example using a set of Linear Learner models that each predict housing prices for a single location. This domain is used as a simple example to easily experiment with multi-model endpoints.  

This notebook showcases three MME capabilities: 
* Native MME support with Amazon SageMaker Linear Learner algorithm.  Because of the native support there is no need for you to create a custom container.  
* Native MME support with Amazon SageMaker Inference Pipelines.
* Granular InvokeModel access to multiple models hosted on the MME using IAM condition key.

To demonstrate these capabilities, the notebook discusses the use case of predicting house prices in multiple cities using linear regression.  House prices are predicted based on features like number of bedrooms, number of garages, square footage etc.  Depending on the city, the features affect the house price differently.  For example, small changes in the square footage cause a drastic change in house prices in New York when compared to price changes in Houston.  For accurate house price predictions, we will train multiple linear regression models, a unique location specific model per city.  


### Contents

1. [Generate synthetic data for housing models](#Generate-synthetic-data-for-housing-models)
1. [Preprocess the raw housing data using Scikit Learn model](#Preprocess-synthetic-housing-data-using-scikit-learn)
1. [Train multiple house value prediction models for multiple cities](#Train-multiple-house-value-prediction-models)
1. [Create model entity with multi model support](#Create-sagemaker-multi-model-support)
1. [Create an inference pipeline with sklearn model and MME linear learner model](#Create-inference-pipeline)
1. [Exercise the inference pipeline - Get predictions from the different  linear learner models](#Exercise-inference-pipeline)
1. [Update Multi Model Endpoint with new models](#update-models)
1. [Explore granular access to the target models of MME](#Finegrain-control-invoke-models)
1. [Endpoint CloudWatch Metrics Analysis](#CW-metric-analysis)
1. [Clean up](#CleanUp)


## Section 1 - Generate synthetic data for housing models <a id='Generate-synthetic-data-for-housing-models'></a>

In this section, you will generate synthetic data that will be used to train the linear learner models.  The data generated consists of 6 numerical features - the year the house was built in, house size in square feet, number of bedrooms, number of bathroom, the lot size and number of garages and two categorial features - deck and front_porch.  

In [2]:
import numpy as np
import pandas as pd
import json
import datetime
import time
import boto3
import sagemaker
import os

from time import gmtime, strftime
from random import choice

from sagemaker import get_execution_role

from sagemaker.multidatamodel import MULTI_MODEL_CONTAINER_MODE
from sagemaker.multidatamodel import MultiDataModel

from sklearn.model_selection import train_test_split

In [3]:
NUM_HOUSES_PER_LOCATION = 1000
LOCATIONS  = ['NewYork_NY',    'LosAngeles_CA',   'Chicago_IL',    'Houston_TX',   'Dallas_TX',
              'Phoenix_AZ',    'Philadelphia_PA', 'SanAntonio_TX', 'SanDiego_CA',  'SanFrancisco_CA']
MAX_YEAR = 2019

In [4]:
def gen_price(house):
    """Generate price based on features of the house"""
    
    if house['FRONT_PORCH'] == 'y':
        garage = 1
    else:
        garage = 0
        
    if house['FRONT_PORCH'] == 'y':
        front_porch = 1
    else:
        front_porch = 0
        
    price = int(150 * house['SQUARE_FEET'] + \
                10000 * house['NUM_BEDROOMS'] + \
                15000 * house['NUM_BATHROOMS'] + \
                15000 * house['LOT_ACRES'] + \
                10000 * garage + \
                10000 * front_porch + \
                15000 * house['GARAGE_SPACES'] - \
                5000 * (MAX_YEAR - house['YEAR_BUILT']))
    return price

In [5]:
def gen_yes_no():
    """Generate values (y/n) for categorical features"""
    answer = choice(['y', 'n'])
    return answer

In [6]:
def gen_random_house():
    """Generate a row of data (single house information)"""
    house = {'SQUARE_FEET':    np.random.normal(3000, 750),
             'NUM_BEDROOMS':  np.random.randint(2, 7),
             'NUM_BATHROOMS': np.random.randint(2, 7) / 2,
             'LOT_ACRES':     round(np.random.normal(1.0, 0.25), 2),
             'GARAGE_SPACES': np.random.randint(0, 4),
             'YEAR_BUILT':    min(MAX_YEAR, int(np.random.normal(1995, 10))),
             'FRONT_PORCH':   gen_yes_no(),
             'DECK':          gen_yes_no()
            }
    
    price = gen_price(house)
    
    return [house['YEAR_BUILT'],   
            house['SQUARE_FEET'], 
            house['NUM_BEDROOMS'], 
            house['NUM_BATHROOMS'], 
            house['LOT_ACRES'],    
            house['GARAGE_SPACES'],
            house['FRONT_PORCH'],    
            house['DECK'], 
            price]

In [7]:
def gen_houses(num_houses):
    """Generate housing dataset"""
    house_list = []
    
    for _ in range(num_houses):
        house_list.append(gen_random_house())
        
    df = pd.DataFrame(
        house_list, 
        columns=[
            'YEAR_BUILT',    
            'SQUARE_FEET',  
            'NUM_BEDROOMS',            
            'NUM_BATHROOMS',
            'LOT_ACRES',
            'GARAGE_SPACES',
            'FRONT_PORCH',
            'DECK', 
            'PRICE']
    )
    return df

In [8]:
def save_data_locally(location, train, test): 
    """Save the housing data locally"""
    os.makedirs('data/{0}/train'.format(location), exist_ok=True)
    train.to_csv('data/{0}/train/train.csv'.format(location), sep=',', header=False, index=False)
       
    os.makedirs('data/{0}/test'.format(location), exist_ok=True)
    test.to_csv('data/{0}/test/test.csv'.format(location), sep=',', header=False, index=False) 

In [12]:
#Generate housing data for multiple locations.
#Change "PARALLEL_TRAINING_JOBS " to a lower number to limit the number of training jobs and models. Or to a higher value to experiment with more models.

PARALLEL_TRAINING_JOBS = 4

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    houses = gen_houses(NUM_HOUSES_PER_LOCATION)
    
    #Spliting data into train and test in 90:10 ratio
    #Not splitting the train data into train and val because its not preprocessed yet
    train, test = train_test_split(houses, test_size=0.1)
    save_data_locally(loc, train, test)


In [13]:
#Shows the first few lines of data.
houses.head()

,YEAR_BUILT,SQUARE_FEET,NUM_BEDROOMS,NUM_BATHROOMS,LOT_ACRES,GARAGE_SPACES,FRONT_PORCH,DECK,PRICE
0,1972,3796.063835,6,2.0,0.74,0,y,y,455509
1,1996,3493.990851,2,3.0,0.74,2,y,y,535198
2,1993,2433.884355,5,2.5,0.88,1,y,n,370782
3,1989,3664.716484,4,2.0,0.67,3,n,n,524757
4,1989,3441.497130,2,2.0,1.36,0,y,y,456624


## Section 2 - Preprocess the raw housing data using Scikit Learn <a id='Preprocess-synthetic-housing-data-using-scikit-learn'></a>

In this section, the categorical features of the data (deck and porch) are pre-processed using sklearn to convert them to one hot encoding representation.  

#### We launch 4 PARALLEL jobs and hence we seemingly create 4 Transformers, but in reality they are just the SAME estimator being run again

In [14]:
%%writefile scripts/sklearn_preprocessor_mce_mme.py
from __future__ import print_function

import argparse
import csv
import json
import os
import shutil
import sys
import time
from io import StringIO

import numpy as np
import pandas as pd
from sagemaker_containers.beta.framework import (
    content_types,
    encoders,
    env,
    modules,
    transformer,
    worker,
)
from sklearn.compose import ColumnTransformer
from sklearn.externals import joblib
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Binarizer, OneHotEncoder, StandardScaler

# Since we get a headerless CSV file we specify the column names here.
feature_columns_names = [
    "YEAR_BUILT",
    "SQUARE_FEET",
    "NUM_BEDROOMS",
    "NUM_BATHROOMS",
    "LOT_ACRES",
    "GARAGE_SPACES",
    "FRONT_PORCH",
    "DECK",
]

label_column = "PRICE"

feature_columns_dtype = {
    "YEAR_BUILT": str,
    "SQUARE_FEET": np.float64,
    "NUM_BEDROOMS": np.float64,
    "NUM_BATHROOMS": np.float64,
    "LOT_ACRES": np.float64,
    "GARAGE_SPACES": np.float64,
    "FRONT_PORCH": str,
    "DECK": str,
}

label_column_dtype = {"PRICE": np.float64}


if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument("--output-data-dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--train", type=str, default=os.environ["SM_CHANNEL_TRAIN"])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [os.path.join(args.train, file) for file in os.listdir(args.train)]
    if len(input_files) == 0:
        raise ValueError(
            (
                "There are no files in {}.\n"
                + "This usually indicates that the train channel was incorrectly specified,\n"
                + "the data specification in S3 was incorrectly specified or the role specified\n"
                + "does not have permission to access the data.".format(args.train)
            )
        )

    for file in input_files:
        print("file :", file)

    raw_data = [pd.read_csv(file, header=None, names=feature_columns_names + [label_column])]

    concat_data = pd.concat(raw_data)

    print(concat_data)

    # This section is adapted from the scikit-learn example of using preprocessing pipelines:
    #
    # https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
    #

    numeric_features = list(feature_columns_names)
    numeric_features.remove("FRONT_PORCH")
    numeric_features.remove("DECK")
    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

    categorical_features = ["FRONT_PORCH", "DECK"]
    categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ],
        remainder="drop",
    )

    preprocessor.fit(concat_data)

    joblib.dump(preprocessor, os.path.join(args.model_dir, "model.joblib"))

    print("saved model!")


def input_fn(input_data, content_type):
    """Parse input data payload

    We currently only take csv input. Since we need to process both labelled
    and unlabelled data we first determine whether the label column is present
    by looking at how many columns were provided.
    """
    if content_type == "text/csv":
        # Read the raw input data as CSV.
        df = pd.read_csv(StringIO(input_data), header=None)

        if len(df.columns) == len(feature_columns_names) + 1:
            # This is a labelled example, includes the ring label
            df.columns = feature_columns_names + [label_column]
        elif len(df.columns) == len(feature_columns_names):
            # This is an unlabelled example.
            df.columns = feature_columns_names

        return df
    else:
        raise ValueError("{} not supported by script!".format(content_type))


def output_fn(prediction, accept):
    """Format prediction output

    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    """
    if accept == "application/json":
        instances = []
        for row in prediction.tolist():
            instances.append({"features": row})

        json_output = {"instances": instances}

        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == "text/csv":
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException("{} accept type is not supported by this script.".format(accept))


def predict_fn(input_data, model):
    """Preprocess input data

    We implement this because the default uses .predict(), but our model is a preprocessor
    so we want to use .transform().

    The output is returned in the following order:

        rest of features either one hot encoded or standardized
    """

    print("Input data type ", type(input_data))

    print(input_data)

    features = model.transform(input_data)

    print("features type ", type(features))

    print(features)

    features_array = features

    print("features_array ", type(features_array))

    print(features_array)

    if label_column in input_data:
        # Return the label (as the first column) and the set of features.
        return np.insert(features_array, 0, input_data[label_column], axis=1)
    else:
        # Return only the set of features
        return features


def model_fn(model_dir):
    """Deserialize fitted model"""
    preprocessor = joblib.load(os.path.join(model_dir, "model.joblib"))
    return preprocessor



Overwriting scripts/sklearn_preprocessor_mce_mme.py


In [15]:
sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
sagemaker_session = sagemaker.Session()

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

BUCKET  = sagemaker_session.default_bucket()
print("BUCKET : ", BUCKET)

role = get_execution_role()
print("ROLE : ", role)

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
REGION = boto3.Session().region_name

DATA_PREFIX = 'DEMO_MME_LINEAR_LEARNER'
HOUSING_MODEL_NAME = 'housing'
MULTI_MODEL_ARTIFACTS = 'multi_model_artifacts'

BUCKET :  sagemaker-us-east-1-622343165275
ROLE :  arn:aws:iam::622343165275:role/service-role/AmazonSageMaker-ExecutionRole-20220208T115633


In [19]:
#Create the SKLearn estimator with the sklearn_preprocessor.py as the script
from sagemaker.sklearn.estimator import SKLearn

script_path = 'scripts/sklearn_preprocessor_mce_mme.py'

sklearn_estimator = SKLearn(
    entry_point=script_path,
    role=role,
    instance_type="ml.c4.xlarge",
    framework_version="0.20.0",
    sagemaker_session=sagemaker_session)

In [20]:
#Upload the raw training data to S3 bucket, to be accessed by SKLearn
train_inputs = []

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:

    train_input = sagemaker_session.upload_data(
        path='data/{}/train/train.csv'.format(loc),
        bucket=BUCKET,
        key_prefix='housing-data/{}/train'.format(loc)
    )
    
    train_inputs.append(train_input)
    print("Raw training data uploaded to : ", train_input)

Raw training data uploaded to :  s3://sagemaker-us-east-1-622343165275/housing-data/NewYork_NY/train/train.csv
Raw training data uploaded to :  s3://sagemaker-us-east-1-622343165275/housing-data/LosAngeles_CA/train/train.csv
Raw training data uploaded to :  s3://sagemaker-us-east-1-622343165275/housing-data/Chicago_IL/train/train.csv
Raw training data uploaded to :  s3://sagemaker-us-east-1-622343165275/housing-data/Houston_TX/train/train.csv


In [21]:
##Launch multiple scikit learn training to process the raw synthetic data generated for multiple locations.
##Before executing this, take the training instance limits in your account and cost into consideration.

sklearn_estimators = []
sklearn_estimator_jobs = []

for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    print("sklearn_estimator fit input data at ", index , " for loc ", loc)
     
    job_name='scikit-learnestimator-{}'.format(strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
    
    sklearn_estimator.fit({'train': train_inputs[index]}, job_name=job_name, wait=False)

    sklearn_estimators.append(sklearn_estimator)
    sklearn_estimator_jobs.append(job_name)
    
    time.sleep(1)

sklearn_estimator fit input data at  0  for loc  NewYork_NY
sklearn_estimator fit input data at  1  for loc  LosAngeles_CA
sklearn_estimator fit input data at  2  for loc  Chicago_IL
sklearn_estimator fit input data at  3  for loc  Houston_TX


In [22]:
#Wait for the preprocessor jobs to finish
for job_name in sklearn_estimator_jobs:
    print('Waiting for job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('training_job_completed_or_stopped')
    waiter.wait(TrainingJobName=job_name)

Waiting for job scikit-learnestimator-2022-08-03-18-30-02 to complete...
Waiting for job scikit-learnestimator-2022-08-03-18-30-03 to complete...
Waiting for job scikit-learnestimator-2022-08-03-18-30-05 to complete...
Waiting for job scikit-learnestimator-2022-08-03-18-30-06 to complete...


In [23]:
##Once the preprocessor is fit, use tranformer to preprocess the raw training data and store the transformed data right back into s3.
##Before executing this, take the training instance limits in your account and cost into consideration.

sklearn_estimator_transformers = []

for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    print("Transform the raw data at ", index , " for loc ", loc)
       
    sklearn_estimator = sklearn_estimators[index]
    
    transformer = sklearn_estimator.transformer(
        instance_count=1,
        instance_type='ml.m4.xlarge',
        assemble_with='Line',
        accept='text/csv'
    )
    
    sklearn_estimator_transformers.append(transformer)

Transform the raw data at  0  for loc  NewYork_NY
Transform the raw data at  1  for loc  LosAngeles_CA
Transform the raw data at  2  for loc  Chicago_IL
Transform the raw data at  3  for loc  Houston_TX


In [66]:
# Preprocess training input
preprocessed_train_data_path = []

for index, transformer in enumerate(sklearn_estimator_transformers):
    transformer.transform(train_inputs[index], content_type='text/csv', wait=False)
    print('STARTING: batch transform job: {}'.format(transformer.latest_transform_job.job_name))
    preprocessed_train_data_path.append(transformer.output_path)

STARTING: batch transform job: sagemaker-scikit-learn-2022-08-03-21-22-08-183
STARTING: batch transform job: sagemaker-scikit-learn-2022-08-03-21-22-08-518
STARTING: batch transform job: sagemaker-scikit-learn-2022-08-03-21-22-09-222
STARTING: batch transform job: sagemaker-scikit-learn-2022-08-03-21-22-10-302


In [67]:
#Wait for all the batch transform jobs to finish
for transformer in sklearn_estimator_transformers: 
    job_name=transformer.latest_transform_job.job_name
    print('Waiting for TRANSFORM job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('transform_job_completed_or_stopped')
    waiter.wait(TransformJobName=job_name)

Waiting for TRANSFORM job sagemaker-scikit-learn-2022-08-03-21-22-08-183 to complete...
Waiting for TRANSFORM job sagemaker-scikit-learn-2022-08-03-21-22-08-518 to complete...
Waiting for TRANSFORM job sagemaker-scikit-learn-2022-08-03-21-22-09-222 to complete...
Waiting for TRANSFORM job sagemaker-scikit-learn-2022-08-03-21-22-10-302 to complete...


In [30]:
##Download the preprocessed data, split into train and val, upload back to S3 in the same directory as tranformer output path
for index, transformer in enumerate(sklearn_estimator_transformers): 
    transformer_output_key='{}/{}'.format(transformer.latest_transform_job.job_name, 'train.csv.out') 
    
    preprocessed_data_download_dir = '{}/'.format("preprocessed-data/"+LOCATIONS[index])
    
    sagemaker_session.download_data(
        path=preprocessed_data_download_dir, 
        bucket=BUCKET,
        key_prefix=transformer_output_key
    )
    
    print('transformer_output_key: {}'.format(transformer_output_key ))
    print('Download directory: {}'.format(preprocessed_data_download_dir ))
    
    train_df = pd.read_csv('{}/{}'.format(preprocessed_data_download_dir,"train.csv.out"))
    
    #Spliting data into train and test in 70:30 ratio
    train, val = train_test_split(train_df, test_size=0.3)
    
    train.to_csv('{}{}'.format(preprocessed_data_download_dir,"train.csv"), sep=',', header=False, index=False)
    val.to_csv('{}{}'.format(preprocessed_data_download_dir,"val.csv"), sep=',', header=False, index=False)
    
    
    train_input = sagemaker_session.upload_data(
        path='{}/{}'.format(preprocessed_data_download_dir, 'train.csv'), 
        bucket=BUCKET,
        key_prefix='{}'.format(transformer.latest_transform_job.job_name, 'train.csv'))
    
    val_input = sagemaker_session.upload_data(
        path='{}/{}'.format(preprocessed_data_download_dir, 'val.csv'), 
        bucket=BUCKET,
        key_prefix='{}'.format(transformer.latest_transform_job.job_name, 'val.csv'))
    
    print(f"Transform:transformer_output_key={transformer_output_key}::Train:uploaded:{train_input}::Validation:{val_input}")   
    
    

transformer_output_key: sagemaker-scikit-learn-2022-08-03-18-34-58-107/train.csv.out
Download directory: preprocessed-data/NewYork_NY/
Transform:transformer_output_key=sagemaker-scikit-learn-2022-08-03-18-34-58-107/train.csv.out::Train:uploaded:s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-34-58-107/train.csv::Validation:s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-34-58-107/val.csv
transformer_output_key: sagemaker-scikit-learn-2022-08-03-18-41-13-787/train.csv.out
Download directory: preprocessed-data/LosAngeles_CA/
Transform:transformer_output_key=sagemaker-scikit-learn-2022-08-03-18-41-13-787/train.csv.out::Train:uploaded:s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-41-13-787/train.csv::Validation:s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-41-13-787/val.csv
transformer_output_key: sagemaker-scikit-learn-2022-08-03-18-47-27-628/train.csv.out
Download directory: pr

In [31]:
##S3 location of the preprocessed data
for preprocessed_train_data in preprocessed_train_data_path: 
    print(preprocessed_train_data)

s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-34-58-107
s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-41-13-787
s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-47-27-628
s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-53-43-093


In [32]:
for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    preprocessed_data_download_dir = '{}/'.format("preprocessed-data/"+LOCATIONS[index])
    path='{}/{}'.format(preprocessed_data_download_dir, 'train.csv')

## Section 3 : Train house value prediction models for multiple cities <a id='Train-multiple-house-value-prediction-models'></a>

In this section, you will use the preprocessed housing data to train multiple linear learner models.

In [33]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='linear-learner')

### Launch a single training job for a given housing location
There is nothing specific to multi-model endpoints in terms of the models it will host. They are trained in the same way as all other SageMaker models. Here we are using the Linear Learner estimator and not waiting for the job to complete.

In [35]:
def launch_training_job(location, transformer):
    """Launch a linear learner traing job"""
    
    train_inputs = '{}/{}'.format(transformer.output_path, "train.csv")
    val_inputs = '{}/{}'.format(transformer.output_path, "val.csv")
    
    print("Training:Job:train_inputs:", train_inputs)
    print("Training:Job:val_inputs:", val_inputs)
     
    full_output_prefix = '{}/model_artifacts/{}'.format(DATA_PREFIX, location)
    s3_output_path = 's3://{}/{}'.format(BUCKET, full_output_prefix)
    
    print("Training:Job:s3_output_path ", s3_output_path)
    
    s3_output_path = 's3://{}/{}/model_artifacts/{}'.format(BUCKET, DATA_PREFIX, location)
    
    linear_estimator = sagemaker.estimator.Estimator(
                            container,
                            role, 
                            instance_count=1,
                            instance_type='ml.c4.xlarge',
                            output_path=s3_output_path,
                            sagemaker_session=sagemaker_session)
    
    linear_estimator.set_hyperparameters(
                           feature_dim=10,
                           mini_batch_size=100,
                           predictor_type='regressor',
                           epochs=10,
                           num_models=32,
                           loss='absolute_loss')
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    train_input = sagemaker.inputs.TrainingInput(s3_data=train_inputs,
                                     distribution=DISTRIBUTION_MODE, content_type='text/csv;label_size=1')
    val_input   = sagemaker.inputs.TrainingInput(s3_data=val_inputs,
                                     distribution=DISTRIBUTION_MODE, content_type='text/csv;label_size=1')
    
    remote_inputs = {'train': train_input, 'validation': val_input}
     
    linear_estimator.fit(remote_inputs, wait=False)
   
    return linear_estimator.latest_training_job.name

### Kick off a model training job for each housing location

In [36]:
training_jobs = []
    
for transformer, loc in zip(sklearn_estimator_transformers, LOCATIONS[:PARALLEL_TRAINING_JOBS]): 
    job = launch_training_job(loc, transformer)
    training_jobs.append(job)
    
print('{} training jobs launched: {}'.format(len(training_jobs), training_jobs))

Training:Job:train_inputs: s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-34-58-107/train.csv
Training:Job:val_inputs: s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-34-58-107/val.csv
Training:Job:s3_output_path  s3://sagemaker-us-east-1-622343165275/DEMO_MME_LINEAR_LEARNER/model_artifacts/NewYork_NY
Training:Job:train_inputs: s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-41-13-787/train.csv
Training:Job:val_inputs: s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-41-13-787/val.csv
Training:Job:s3_output_path  s3://sagemaker-us-east-1-622343165275/DEMO_MME_LINEAR_LEARNER/model_artifacts/LosAngeles_CA
Training:Job:train_inputs: s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-47-27-628/train.csv
Training:Job:val_inputs: s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-03-18-47-27-628/val.csv
Training:Job:s3_output_path  s3://sagemak

### Wait for all  training jobs to finish

In [38]:
#Wait for the jobs to finish
for job_name in training_jobs:
    print('Waiting for Training:job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('training_job_completed_or_stopped')
    waiter.wait(TrainingJobName=job_name)

Waiting for Training:job linear-learner-2022-08-03-19-06-48-806 to complete...
Waiting for Training:job linear-learner-2022-08-03-19-06-49-213 to complete...
Waiting for Training:job linear-learner-2022-08-03-19-06-52-720 to complete...
Waiting for Training:job linear-learner-2022-08-03-19-06-53-084 to complete...


## Section 4 - Create Sagemaker model with multi model support <a id='Create-sagemaker-multi-model-support'></a>

In [39]:
import re
def parse_model_artifacts(model_data_url):
    # extract the s3 key from the full url to the model artifacts
    s3_key = model_data_url.split('s3://{}/'.format(BUCKET))[1]
    # get the part of the key that identifies the model within the model artifacts folder
    model_name_plus = s3_key[s3_key.find('model_artifacts') + len('model_artifacts') + 1:]
    # finally, get the unique model name (e.g., "NewYork_NY")
    model_name = re.findall('^(.*?)/', model_name_plus)[0]
    return s3_key, model_name 

In [40]:
# make a copy of the model artifacts from the original output of the training job to the place in
# s3 where the multi model endpoint will dynamically load individual models
def copy_artifacts_to_mme(job_name):
    print("job_name :", job_name)
    response = sm_client.describe_training_job(TrainingJobName=job_name)
    source_s3_key, model_name = parse_model_artifacts(response['ModelArtifacts']['S3ModelArtifacts'])
    copy_source = {'Bucket': BUCKET, 'Key': source_s3_key}
    key = '{}/{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name, model_name)
    
    print('Copying {} model\n   from: {}\n     to: {}...'.format(model_name, source_s3_key, key))
    s3_client.copy_object(Bucket=BUCKET, CopySource=copy_source, Key=key)


In [41]:
# First, clear out old versions of the model artifacts from previous runs of this notebook
s3_bucket = s3.Bucket(BUCKET)
full_input_prefix = '{}/multi_model_artifacts'.format(DATA_PREFIX)
print('Removing old model artifacts from {}'.format(full_input_prefix))
s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

Removing old model artifacts from DEMO_MME_LINEAR_LEARNER/multi_model_artifacts


[]

In [42]:
## Deploy all but the last model trained to MME
## We will use the last model to show how to update an existing MME in Section 7
for job_name in training_jobs[:-1]:
    copy_artifacts_to_mme(job_name)

job_name : linear-learner-2022-08-03-19-06-48-806
Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/NewYork_NY/linear-learner-2022-08-03-19-06-48-806/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz...
job_name : linear-learner-2022-08-03-19-06-49-213
Copying LosAngeles_CA model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/LosAngeles_CA/linear-learner-2022-08-03-19-06-49-213/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/LosAngeles_CA/LosAngeles_CA.tar.gz...
job_name : linear-learner-2022-08-03-19-06-52-720
Copying Chicago_IL model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/Chicago_IL/linear-learner-2022-08-03-19-06-52-720/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Chicago_IL/Chicago_IL.tar.gz...


In [43]:
MODEL_NAME = '{}-{}'.format(HOUSING_MODEL_NAME, strftime('%Y-%m-%d-%H-%M-%S', gmtime()))

_model_url  = 's3://{}/{}/{}/'.format(BUCKET, DATA_PREFIX, MULTI_MODEL_ARTIFACTS)
print(_model_url)
ll_multi_model = MultiDataModel(
        name=MODEL_NAME,
        model_data_prefix=_model_url,
        image_uri=container,
        role=role,
        sagemaker_session=sagemaker_session
    )

s3://sagemaker-us-east-1-622343165275/DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/


## Section 4b: Show the Multi Model structure

The structure of the Multi Container and Multi Model is as follows:

Multi Model Container

in S3 location -  

DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/

    |
    | ----- Chicago_IL
    |          |
    |          | ---  Model.tar.gz
    | ------ NewYork_NY
    |          |
    |          | ---  Model.tar.gz
    
    
Container MME points to  ----- >  DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/  as the top level container

Invoke Container MME is  ----- >  Chicago_IL/Chicago_IL.tar.gz

This however will pass through via the 1st Container which is a SCIKIT-Learn Pre processor MODEL so sequence becomes

Client --- > Scit-learn-preprocessorModel --- > MME server --- > Call Chicago Model --- > Return results 

In [44]:
print(_model_url)
! aws s3 ls {_model_url}

s3://sagemaker-us-east-1-622343165275/DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/
                           PRE Chicago_IL/
                           PRE LosAngeles_CA/
                           PRE NewYork_NY/


## Section 5 : Create an inference pipeline with sklearn model and MME linear learner model <a id='Create-inference-pipeline'></a>

Set up the inference pipeline using the Pipeline Model API.  This sets up a list of models in a single endpoint; In this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model.

In [45]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_estimator.create_model()

model_name = '{}-{}'.format('inference-pipeline', timestamp_prefix)
endpoint_name = '{}-{}'.format('inference-pipeline-ep', timestamp_prefix)

print(model_name)
print(endpoint_name)

inference-pipeline-2022-08-03-19-20-46
inference-pipeline-ep-2022-08-03-19-20-46


In [46]:
sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    sagemaker_session=sagemaker_session,
    models=[
        scikit_learn_inference_model, 
        ll_multi_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

---------------------!

## Section 6 :  Exercise the inference pipeline - Get predictions from  different  linear learner models. <a id='Exercise-inference-pipeline'></a>

In [47]:
#Create Predictor
from sagemaker.predictor import Predictor

csv_serializer = sagemaker.serializers.CSVSerializer()

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)

In [48]:
def predict_one_house_value(features, model_name, predictor_to_use):
    print('Using model {} to predict price of this house: {}'.format(model_name,
                                                                     features))
    body = ','.join(map(str, features)) + '\n'
    start_time = time.time()
     
    response = predictor_to_use.predict(features, target_model=model_name)
    
    response_json = json.loads(response)
        
    predicted_value = response_json['predictions'][0]['score']    
    
    duration = time.time() - start_time
    
    print('${:,.2f}, took {:,d} ms\n'.format(predicted_value, int(duration * 1000)))

## Pipeline Model
Since we have the Pipeline Model -- it will convert the values to Numerical and then pass to the Linear Learner

In [49]:
for _ in range(10):
    model_name = LOCATIONS[np.random.randint(1, PARALLEL_TRAINING_JOBS - 1)]
    full_model_name = '{}/{}.tar.gz'.format(model_name,model_name)
    predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict price of this house: [2013, 3908.3702266351775, 5, 3.0, 1.1, 0, 'n', 'n']
$666,358.06, took 3,312 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [1985, 3304.24213565395, 5, 2.0, 1.58, 2, 'y', 'y']
$480,957.22, took 1,707 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [1989, 3334.054136567468, 6, 1.5, 1.37, 2, 'y', 'n']
$503,117.50, took 49 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [1992, 1440.8661169803904, 3, 1.0, 1.04, 0, 'n', 'n']
$144,057.62, took 51 ms

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict price of this house: [1984, 2613.3388042867655, 3, 2.0, 1.06, 1, 'y', 'y']
$334,407.84, took 39 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [2008, 2005.5694198700348, 6, 3.0, 1.22, 0, 'y', 'n']
$393,485.88, took 37 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [

## Section 7 - Add new model to the endpoint, simply by copying the model artifact to the S3 location
<a id='update-models'></a>

In [50]:
## Copy the last model
last_training_job=training_jobs[PARALLEL_TRAINING_JOBS-1]
copy_artifacts_to_mme(last_training_job)

job_name : linear-learner-2022-08-03-19-06-53-084
Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/Houston_TX/linear-learner-2022-08-03-19-06-53-084/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz...


In [51]:
model_name = LOCATIONS[PARALLEL_TRAINING_JOBS-1]
full_model_name = '{}/{}.tar.gz'.format(model_name,model_name)
predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)

Using model Houston_TX/Houston_TX.tar.gz to predict price of this house: [1992, 2831.3574471631478, 6, 1.5, 0.98, 1, 'y', 'y']
$429,373.12, took 1,691 ms



#### Call again to demo the second call wil be a lot faster

In [54]:
model_name = 'Houston_TX'
full_model_name = '{}/{}.tar.gz'.format(model_name,model_name)
predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)

Using model Houston_TX/Houston_TX.tar.gz to predict price of this house: [2004, 4786.370765689304, 5, 3.0, 1.04, 2, 'y', 'y']
$807,585.75, took 42 ms



## Section 8 - Endpoint CloudWatch Metrics Analysis <a id='CW-metric-analysis'></a>

With MME, the models are dynamically loaded into the container’s memory of the instance hosting the endpoint when invoked.  Therefore, the model invocation may take longer when it is invoked for the first time. And after the model is already in the container’s memory, the subsequent invocations will be faster. If an instance memory utilization is high and a new model needs to be loaded then unused models are unloaded.  The unloaded models will remain in the instance’s storage volume and can be loaded into container’s memory later without being downloaded from the S3 bucket again.  If the instance’s storage volume if full, unused models are deleted from storage volume.    
Managing the loading/unloading of the models is completely handled by Amazon SageMaker behind the scenes without you having to take any specific actions.  However, it is important to understand this behavior because it has implications on the model invocation latency.

Amazon SageMaker provides CloudWatch metrics for multi-model endpoints so you can determine the endpoint usage and the cache hit rate and optimize your endpoint.  To analyze the endpoint and the container behavior, you will invoke multiple models in this order :

    a. Create 200 copies of the original model and save with different names.
    b. Starting with no models loaded into the container, Invoke the first 100 models
    c. Invoke the same 100 models again
    d. Invoke all 200 models

We use this order of invocations to observe the behavior of the CloudWatch metrics - LoadedModelCount, MemoryUtilization and ModelCacheHit.  You are encouraged to experiment with loading varying number of models to use the CloudWatch charts to help make ongoing decisions on the optimal choice of instance type, instance count, and number of models that a given endpoint should host.



In [56]:
# Make a copy of the model artifacts in S3 bucket with new names so we have multiple models to understand the latency behavior.
def copy_additional_artifacts_to_mme(num_copies):
    
    source_s3_model_key = '{}/{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name, model_name)
    _copy_source = {'Bucket': BUCKET, 'Key': source_s3_model_key}
    for i in range(num_copies):
        new_model_name="{}_{}".format(i, model_name)
        dest_s3_model_key = '{}/{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name, new_model_name)
        print('Copying {} model\n   from: {}\n     to: {}...'.format(model_name, source_s3_model_key, dest_s3_model_key))
        s3_client.copy_object(Bucket=BUCKET, CopySource=_copy_source, Key=dest_s3_model_key)

In [59]:
model_name='NewYork_NY'

In [60]:
##Create 200 copies of the original model and save with different names.
copy_additional_artifacts_to_mme(200)

Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/0_NewYork_NY.tar.gz...
Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/1_NewYork_NY.tar.gz...
Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/2_NewYork_NY.tar.gz...
Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/3_NewYork_NY.tar.gz...
Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/4_NewYork_NY.tar.gz...
Copying Ne

In [61]:
##Invoke multiple models in a loop
def invoke_multiple_models_mme(model_range_low, model_range_high):
    for i in range(model_range_low, model_range_high):
        new_model_name="{}_{}".format(i, model_name)
        full_model_name = '{}/{}.tar.gz'.format(model_name, new_model_name)
        predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)


In [62]:
##Starting with no models loaded into the container
##Invoke the first 100 models
invoke_multiple_models_mme(10, 19)

Using model NewYork_NY/10_NewYork_NY.tar.gz to predict price of this house: [2001, 2193.637294405955, 5, 2.0, 1.06, 1, 'n', 'y']
$352,423.53, took 1,741 ms

Using model NewYork_NY/11_NewYork_NY.tar.gz to predict price of this house: [1997, 3575.971674100197, 2, 2.5, 1.3, 2, 'y', 'y']
$552,953.19, took 1,634 ms

Using model NewYork_NY/12_NewYork_NY.tar.gz to predict price of this house: [1985, 2874.7334503329557, 5, 2.0, 0.98, 3, 'y', 'y']
$421,522.47, took 1,692 ms

Using model NewYork_NY/13_NewYork_NY.tar.gz to predict price of this house: [1983, 3708.3163654589825, 5, 1.5, 1.23, 3, 'n', 'n']
$515,408.91, took 1,649 ms

Using model NewYork_NY/14_NewYork_NY.tar.gz to predict price of this house: [2010, 3097.271995579226, 4, 1.0, 0.9, 3, 'y', 'n']
$557,202.06, took 1,621 ms

Using model NewYork_NY/15_NewYork_NY.tar.gz to predict price of this house: [2003, 2766.141238183692, 5, 2.5, 0.95, 3, 'n', 'n']
$485,428.88, took 1,651 ms

Using model NewYork_NY/16_NewYork_NY.tar.gz to predict pri

In [63]:
##Invoke the same 100 models again
invoke_multiple_models_mme(10, 19)

Using model NewYork_NY/10_NewYork_NY.tar.gz to predict price of this house: [1992, 3632.5207788432867, 5, 1.0, 0.85, 1, 'y', 'y']
$524,681.25, took 38 ms

Using model NewYork_NY/11_NewYork_NY.tar.gz to predict price of this house: [1997, 3139.5808729929013, 2, 2.0, 1.0, 3, 'y', 'n']
$489,719.56, took 40 ms

Using model NewYork_NY/12_NewYork_NY.tar.gz to predict price of this house: [2005, 2662.030022745317, 4, 3.0, 1.05, 1, 'y', 'n']
$468,263.94, took 36 ms

Using model NewYork_NY/13_NewYork_NY.tar.gz to predict price of this house: [1987, 2579.1510852184533, 3, 2.5, 1.4, 1, 'y', 'n']
$350,861.53, took 35 ms

Using model NewYork_NY/14_NewYork_NY.tar.gz to predict price of this house: [1989, 2200.3373029245263, 2, 1.5, 1.25, 3, 'y', 'n']
$304,390.41, took 36 ms

Using model NewYork_NY/15_NewYork_NY.tar.gz to predict price of this house: [1987, 2152.7593078109876, 4, 2.5, 1.25, 0, 'y', 'n']
$280,207.75, took 36 ms

Using model NewYork_NY/16_NewYork_NY.tar.gz to predict price of this hous

In [64]:
##This time invoke all 200 models to observe behavior
invoke_multiple_models_mme(0, 200)

Using model NewYork_NY/0_NewYork_NY.tar.gz to predict price of this house: [2002, 3122.05526662563, 5, 2.5, 0.78, 0, 'y', 'y']
$504,932.66, took 1,842 ms

Using model NewYork_NY/1_NewYork_NY.tar.gz to predict price of this house: [2004, 3283.426705278836, 5, 1.5, 1.07, 2, 'y', 'n']
$561,967.25, took 1,685 ms

Using model NewYork_NY/2_NewYork_NY.tar.gz to predict price of this house: [1991, 3095.6658181328676, 4, 2.0, 1.11, 3, 'n', 'n']
$457,362.34, took 1,675 ms

Using model NewYork_NY/3_NewYork_NY.tar.gz to predict price of this house: [1979, 4121.453296555162, 2, 1.5, 0.83, 3, 'y', 'n']
$534,163.69, took 1,626 ms

Using model NewYork_NY/4_NewYork_NY.tar.gz to predict price of this house: [1990, 3288.091413322445, 3, 1.0, 0.38, 3, 'n', 'n']
$439,780.09, took 1,671 ms

Using model NewYork_NY/5_NewYork_NY.tar.gz to predict price of this house: [1993, 2824.8616224183243, 6, 2.5, 1.32, 3, 'y', 'n']
$482,573.84, took 1,850 ms

Using model NewYork_NY/6_NewYork_NY.tar.gz to predict price of 

#### CloudWatch charts for LoadedModelCount,MemoryUtilization and ModelCacheHit metrics will be similar to charts below.

![](cw_charts/ModelCountMemUtilization.png)

“LoadedModelCount” continuously increases, as more models are invoked, till it levels off at 121.  “MemoryUtilization” of the container also increased correspondingly to around 79%.  This shows that the instance chosen to host the endpoint, could only maintain 121 models in memory, when 200 model invocations are made.  

![](cw_charts/ModelCountMemUtilizationCacheHit.png)

As the number of models loaded to the container memory increase, the ModelCacheHit improves.  When the same 100 models are invoked the second time, the ModelCacheHit reaches 1.  When new models, not yet loaded are invoked the ModelCacheHit decreases again. 

## Section 9 - Explore granular access to the target models of MME <a id='Finegrain-control-invoke-models'></a>

If the role attached to this notebook instance allows invoking SageMaker endpoints, it is able to invoke all models hosted on the MME.  Using IAM conditional keys, you can restrict this model invocation access to specific models.  To explore this, you will create a new IAM role and IAM policy with conditional key to restrict access to a single model.  Assume this new role and verify that only a single target model can be invoked.

Note that to execute this section, the role attached to the notebook instance should allow the following actions :
    "iam:CreateRole",
    "iam:CreatePolicy",
    "iam:AttachRolePolicy",
    "iam:UpdateAssumeRolePolicy"
    
If this is not the case, please work with the Administrator of this AWS account to ensure this.  

In [ ]:
iam_client = boto3.client('iam')

In [ ]:
#Create a new role that can be assumed by this notebook.  The roles should allow access to only a single model.

path='/'

role_name="{}{}".format('allow_invoke_ny_model_role', strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
description='Role that allows invoking a single model'

action_string = "sts:AssumeRole"
    
trust_policy={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "statement1",
      "Effect": "Allow",
      "Principal": {
        "AWS": role
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [ ]:
response = iam_client.create_role(
    Path=path,
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(trust_policy),
    Description=description,
    MaxSessionDuration=3600
)

In [ ]:
role_arn=response['Role']['Arn']
print("Role arn is :", role_arn)

In [ ]:
endpoint_resource_arn = "arn:aws:sagemaker:{}:{}:endpoint/{}".format(REGION, ACCOUNT_ID, endpoint_name)
print("Endpoint arn is :", endpoint_resource_arn)

In [ ]:
##Create the IAM policy with the IAM condition key
policy_name = "{}{}".format('allow_invoke_ny_model_policy', strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
managed_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "SageMakerAccess",
            "Action": "sagemaker:InvokeEndpoint",
            "Effect": "Allow",
            "Resource":endpoint_resource_arn,
            "Condition": {
                "StringLike": {
                    "sagemaker:TargetModel": ["NewYork_NY/*"]
                }
            }
        }
    ]
}

response = iam_client.create_policy(
  PolicyName=policy_name,
  PolicyDocument=json.dumps(managed_policy)
)

In [ ]:
policy_arn=response['Policy']['Arn']

In [ ]:
##Attach policy to role
iam_client.attach_role_policy(
    PolicyArn=policy_arn,
    RoleName=role_name
)

In [ ]:
## Invoke with the role that has access to only NY model
sts_connection = boto3.client('sts')
assumed_role_limited_access = sts_connection.assume_role(
    RoleArn=role_arn,
    RoleSessionName="MME_Invoke_NY_Model"
)
assumed_role_limited_access['AssumedRoleUser']['Arn']


In [ ]:
trust_policy={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "statement1",
      "Effect": "Allow",
      "Principal": {
        "AWS": role
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Sid": "statement2",
      "Effect": "Allow",
      "Principal": {
          "AWS": assumed_role_limited_access['AssumedRoleUser']['Arn']
      },
      "Action": "sts:AssumeRole"
    }  
  ]
}

In [ ]:
iam_client.update_assume_role_policy(
    RoleName=role_name,
    PolicyDocument=json.dumps(trust_policy)
)

In [ ]:
ACCESS_KEY = assumed_role_limited_access['Credentials']['AccessKeyId']
SECRET_KEY = assumed_role_limited_access['Credentials']['SecretAccessKey']
SESSION_TOKEN = assumed_role_limited_access['Credentials']['SessionToken']

runtime_sm_client_with_assumed_role = boto3.client(
    service_name='sagemaker-runtime', 
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN,
)

In [ ]:
 sagemakerSessionAssumedRole = sagemaker.Session(sagemaker_runtime_client=runtime_sm_client_with_assumed_role)

In [ ]:
predictorAssumedRole = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemakerSessionAssumedRole,
    serializer=csv_serializer)

In [ ]:
full_model_name = 'NewYork_NY/NewYork_NY.tar.gz'
predict_one_house_value(gen_random_house()[:-1], full_model_name,predictorAssumedRole)

In [ ]:
##This should fail with "AccessDeniedException" since the assumed role does not have access to Chicago model
full_model_name = 'Chicago_IL/Chicago_IL.tar.gz'
predict_one_house_value(gen_random_house()[:-1], full_model_name,predictorAssumedRole)

## Clean up<a id='CleanUp'></a>
Clean up the endpoint to avoid unneccessary costs.



In [ ]:
#Delete the endpoint and underlying model
predictor.delete_model() 
predictor.delete_endpoint()
for t in preprocessor_transformers:
    t.delete_model()

In [ ]:
#Delete the IAM Role
iam_client.detach_role_policy(
    PolicyArn=policy_arn,
    RoleName=role_name
)
iam_client.delete_role(RoleName=role_name)

In [ ]:
#Delete the IAM Policy
iam_client.delete_policy(PolicyArn=policy_arn)